### Modeling video data with 3DCNN for frame and GRU for time

Need output data from <b>01 processing videos</b>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GRU, Attention,Flatten, Reshape, Concatenate
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPool2D, MaxPool3D, Conv1D, MaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as kr
import pickle
from data_split import train_valid_test_split

### Load data

In [ ]:
####
#parameters
test_rate = 0.15
valid_rate = 0.15
window = 30 #frame window
seed = 123456

####
#load focus videos
of = open('processed_videos_160.obj', 'rb')
data, labels = pickle.load(of)
of.close()

####
#transform classes
from sklearn.preprocessing import OrdinalEncoder
label_nums = OrdinalEncoder().fit_transform(labels)

####
#split into train/valid/test
X_train,X_valid,X_test,y_train,y_valid,y_test = train_valid_test_split(data,label_nums,test_rate,valid_rate,window,seed)

### Model building

In [ ]:
####
#parameters
face_cnn_layers = [[8,2,4,4], [4,2,4,4], [4,2,4,4]]
gru_size = 64
out_dense = [64]
dropout = False
droprate = 0.4

####
#functional codes
face_in = tf.keras.Input(shape=(window,160,160,1))
x = face_in
#Face 3dCNN blocks
for fcnn in face_cnn_layers:
    x = Conv3D(fcnn[0],(fcnn[1],fcnn[2],fcnn[3]), activation="relu")(x)
    x = MaxPool3D([2,3,3], padding='same')(x)
    x = BatchNormalization()(x)
    if dropout:
        x = Dropout(droprate)(x)
x = Reshape((x.shape[1],x.shape[2]*x.shape[3]*x.shape[4]))(x)
# RNN Block
x = GRU(gru_size, return_sequences=True)(x)
x = BatchNormalization()(x)
if dropout:
    x = Dropout(droprate)(x)
# Self-Attention Block
x = Attention()([x, x])
x = tf.keras.layers.GlobalMaxPooling1D()(x)
# Output Block
for d in out_dense:
    x = Dense(d, activation="relu")(x)
    if dropout:
        x = Dropout(droprate)(x)
output   = Dense(units=1, activation='softmax')(x)
# Build and Compile model
model = Model(inputs=[face_in], outputs=[output])
model.summary()

### Training

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=[X_valid, y_valid], batch_size=192, epochs=1000)

### Testing

In [ ]:
model.evaluate(X_test_face, y_test)